# AutoKeras Model Generator

# Imports

In [1]:
import os
import numpy as np
%matplotlib inline
import matplotlib.pylab as plt
import autokeras as ak

#pip install absl-py
#tensorflow 2.3.0 is not installed properly 


# Use the 0th GPU in the computer
os.environ["CUDA_VISIBLE_DEVICES"]="0"


#import tensorflow as tf
#tf.config.list_physical_devices('GPU')

# Functions

In [2]:
def norma2(input_data):
    """Normalize all data based on their own max values
    
    Parameters
    ==========
    
    input_data - (np.array)
        the full array of all 2D images to normalize based upon their own np.max value
        
        
    Returns
    =======
    The normalized data
    
    """
    
    maxies = np.max(input_data, axis=(1,2))
    if np.count_nonzero(maxies) != len(input_data):
        print('Error - Divide By Zero Found')
    output = np.divide(input_data, maxies[:,None, None])

    
    return output


In [3]:
# Load in the already split Training and Test Sets 64x64 image dataset
xTrain = np.load('../training_data/noise_train_datatset_01.npy', allow_pickle=True)
yTrain = np.load('../training_data/noise_train_labels_01.npy', allow_pickle=True)

# DataPrep

In [4]:
# Obtain every 8th data point for faster training
new_xTrain2 = []
new_yTrain2 = []

for kk, value in enumerate(xTrain):
    if kk%8 == 0:
        new_xTrain2.append(value)
        new_yTrain2.append(yTrain[kk])
        
new_xTrain2 = np.asarray(new_xTrain2)
new_yTrain2 = np.asarray(new_yTrain2)

xTrain = new_xTrain2
yTrain = new_yTrain2

new_xTrain2.shape, new_yTrain2.shape

((22032, 64, 64), (22032, 3))

In [5]:
# Section out Perfect - Edge - Screw - Training Testing
perf_dat = []
edge_dat = []
screw_dat = []

perf_lab = []
edge_lab = []
screw_lab = []


for idx, labels in enumerate(yTrain):
    if labels[0] == 1:
        perf_lab.append(labels)
        perf_dat.append(xTrain[idx])
    
    elif labels[1] == 1:
        edge_lab.append(labels)
        edge_dat.append(xTrain[idx])
        
    elif labels[2] == 1:
        screw_lab.append(labels)
        screw_dat.append(xTrain[idx])

        
perf_dat = np.asarray(perf_dat)
edge_dat = np.asarray(edge_dat)
screw_dat = np.asarray(screw_dat)

perf_lab = np.asarray(perf_lab)
edge_lab = np.asarray(edge_lab)
screw_lab = np.asarray(screw_lab)


X_train = np.vstack((edge_dat, screw_dat))
y_train = np.vstack((edge_lab, screw_lab))

print('ONLY HAS EDGE AND SCREW DATA!!!!')


from sklearn.model_selection import train_test_split
xTrain, xTest, yTrain, yTest = train_test_split(X_train, 
                                                y_train, 
                                                test_size = 0.15, 
                                                random_state = 0)

print(xTrain.shape, xTrain.dtype, xTest.shape, xTest.dtype)

ONLY HAS EDGE AND SCREW DATA!!!!
(12484, 64, 64) float64 (2204, 64, 64) float64


In [6]:
y_train = np.asarray(yTrain)
y_test = np.asarray(yTest)

X_train = norma2(xTrain)
X_test = norma2(xTest)


X_train = X_train.reshape((X_train.shape[0], 64, 64, 1))
X_test = X_test.reshape((X_test.shape[0], 64, 64, 1))

X_train = np.asarray(np.repeat(X_train, 3, 3))
X_test = np.asarray(np.repeat(X_test, 3, 3))


X_train.shape, y_train.shape, X_test.shape, y_test.shape, X_train.dtype, X_test.dtype

((12484, 64, 64, 3),
 (12484, 3),
 (2204, 64, 64, 3),
 (2204, 3),
 dtype('float64'),
 dtype('float64'))

# Build AutoKeras

In [8]:
clf = ak.ImageClassifier(max_trials=100, overwrite=False, tuner='bayesian',)
clf.fit(X_train, y_train, epochs=9, verbose=1, )

# Sometimes this does not return the best model. Use code below to obtain
    # all models produced
#model = clf.export_model()

Trial 3 Complete [00h 08m 50s]
val_loss: 0.8166293501853943

Best val_loss So Far: 0.42778176069259644
Total elapsed time: 01h 09m 56s

Search: Running Trial #4

Hyperparameter      |Value     |Best Value So Far   
image_block_1/normalize|False     |True                
image_block_1/augment|False     |False               
image_block_1/block_type|resnet    |vanilla             
image_block_1/res_net_block_1/pretrained|True      |None                
image_block_1/res_net_block_1/version|resnet101 |None                
image_block_1/res_net_block_1/imagenet_size|False     |None                
classification_head_1/spatial_reduction_1/reduction_type|flatten   |flatten             
classification_head_1/dropout|0         |0.5                 
optimizer           |adam      |adam                
learning_rate       |0.001     |0.001               

171450368/171446536 [==============================] - 26s 0us/step


Traceback (most recent call last):
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/autokeras-1.0.6.dev0-py3.7.egg/autokeras/graph.py", line 263, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/autokeras-1.0.6.dev0-py3.7.egg/autokeras/engine/block.py", line 48, in _build_wrapper
    return super()._build_wrapper(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/si

Invalid model 0/5


Traceback (most recent call last):
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/autokeras-1.0.6.dev0-py3.7.egg/autokeras/graph.py", line 263, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/autokeras-1.0.6.dev0-py3.7.egg/autokeras/engine/block.py", line 48, in _build_wrapper
    return super()._build_wrapper(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/si

Invalid model 1/5


...

Traceback (most recent call last):
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/autokeras-1.0.6.dev0-py3.7.egg/autokeras/graph.py", line 263, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/autokeras-1.0.6.dev0-py3.7.egg/autokeras/engine/block.py", line 48, in _build_wrapper
    return super()._build_wrapper(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/si

Invalid model 4/5


Traceback (most recent call last):
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 104, in build
    model = self.hypermodel.build(hp)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/autokeras-1.0.6.dev0-py3.7.egg/autokeras/graph.py", line 263, in build
    outputs = block.build(hp, inputs=temp_inputs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/autokeras-1.0.6.dev0-py3.7.egg/autokeras/engine/block.py", line 48, in _build_wrapper
    return super()._build_wrapper(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/site-packages/kerastuner/engine/hypermodel.py", line 64, in _build_wrapper
    return self._build(hp, *args, **kwargs)
  File "/home/will/anaconda3/envs/autok2/lib/python3.7/si

Invalid model 5/5


RuntimeError: Too many failed attempts to build model.

In [ ]:
out = clf.tuner.get_best_models(100)
for i, mm in enumerate(out):
    mm.save(f'../intermediate_dir/autokeras_models/{i}')